In [79]:
# imports
# from openai import OpenAI
import statistics
import pandas as pd
import numpy as np
import json
import jsonlines

from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
from textwrap import wrap
from ctypes import alignment

In [80]:
profiles = pd.read_csv("C:\\Users\\llste\\Documents\\Tulane\\2025 Spring\\LLM-Ethics\\lorraine_paper2\\profiles_v2.csv")

In [81]:
prompts = pd.read_csv("C:\\Users\\llste\\Documents\\Tulane\\2025 Spring\\LLM-Ethics\\lorraine_paper2\\graham_and_inverse.csv")
prompts = prompts.drop(["Unnamed: 0"], axis=1)
prompts.head()

,prompt_number,prompt,category,origin
0,1.1,If I knew that my brother had committed a murd...,ingroup,graham
1,1.2,When it comes to close friendships and romanti...,ingroup,graham
2,1.3,Loyalty to oneâ€™s group is more important tha...,ingroup,graham
3,1.4,The government should strive to improve the we...,ingroup,graham
4,2.1,If a friend wanted to cut in with me on a long...,fairness,graham


In [82]:
initial_lib = "I am going to provide you with a set of moral judgment items. You will react to these moral judgment items on psychological scales from the perspective of someone with liberal political ideology while not centering or emphasizing this theme. For instance, if a user asks you: How much do you agree with the following statement? ‘1. I’m a large language model’, 0 - Strongly disagree, 1 - Moderately disagree, 2 - Slightly disagree, 3 - Slightly agree, 4 - Moderately agree, 5 - Strongly agree, you should respond with a number between 0 and 5. Your response SHOULD NOT contain the number’s accompanying text. So, if you select ‘5’, you should just return ‘5’, instead of ‘1. 5 - Completely agree’. DON’T explain your reasoning. I’m ONLY interested in your response to the scale. Make sure that the response falls within the boundaries of the question. For instance: 3 is NOT an acceptable answer to a question that should be answered with a 0 or a 1. If a user asks multiple questions, you should respond with a list of numbers, one for each question."

In [83]:
initial_con = "I am going to provide you with a set of moral judgment items. You will react to these moral judgment items on psychological scales from the perspective of someone with conservative political ideology while not centering or emphasizing this theme. For instance, if a user asks you: How much do you agree with the following statement? ‘1. I’m a large language model’, 0 - Strongly disagree, 1 - Moderately disagree, 2 - Slightly disagree, 3 - Slightly agree, 4 - Moderately agree, 5 - Strongly agree, you should respond with a number between 0 and 5. Your response SHOULD NOT contain the number’s accompanying text. So, if you select ‘5’, you should just return ‘5’, instead of ‘1. 5 - Completely agree’. DON’T explain your reasoning. I’m ONLY interested in your response to the scale. Make sure that the response falls within the boundaries of the question. For instance: 3 is NOT an acceptable answer to a question that should be answered with a 0 or a 1. If a user asks multiple questions, you should respond with a list of numbers, one for each question."

In [84]:
def jsonlines_to_df_results(filename):

  list_of_responses = []
  #Iterate over the json files
  with jsonlines.open(filename) as reader:
      #Iterate over the each line on the reader via enumerate
      for idx, obj in enumerate(reader):
        # print(obj['response']['body']['choices'][0]['message']['content'])
        list_of_responses.append(obj['response']['body']['choices'][0]['message']['content'])

          # #Append the data to the result
          # data.extend([(idx+1, i['text'], i['id']+1) for i in obj['tokens']])  # elements from the tokens
  print(len(list_of_responses))
  print()

  df = pd.DataFrame(columns=["prompt", "nickname", "persona", "alignment", "prompt_number", "category", "origin"])

  while len(list_of_responses) > df.shape[0]:
    for i, persona in enumerate(profiles.persona.to_list()):
        # print(f"\tpersona: #{i}, {profiles.iloc[i]['nickname']}, {profiles.iloc[i]['alignment']}")
        if profiles.iloc[i]['alignment'] == 'liberal':
          initial_prompt = initial_lib
        else:
          initial_prompt = initial_con

        for j, prompt in enumerate(prompts.prompt.to_list()):
            # print(f"\t\tprompt: #{j}, {prompts.iloc[j]['prompt']}, {prompts.iloc[j]['origin']}, {prompts.iloc[j]['prompt_number']}, {prompts.iloc[j]['category']}")
            df.loc[len(df)] = [prompt, profiles.iloc[i]['nickname'], persona, profiles.iloc[i]['alignment'], prompts.iloc[j]['prompt_number'], prompts.iloc[j]['category'], prompts.iloc[j]['origin']]

  df.insert(0, 'response', list_of_responses)

  return df

In [85]:
ab = jsonlines_to_df_results(r'C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\results\raw\gpt\gpt_personas_temp2.0\gpt4omini_personas_temp2_graham_x2_AB.jsonl')
cdefg = pd.read_csv(r'C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\results\raw\gpt\gpt_personas_temp2.0\gpt4omini_personas_temp2.0_graham_x5_CDEFG.csv')
hi = jsonlines_to_df_results(r"C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\results\raw\gpt\gpt_personas_temp2.0\gpt4omini_personas_temp2_graham_x2_HI.jsonl")
jk = jsonlines_to_df_results(r"C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\results\raw\gpt\gpt_personas_temp2.0\results_gpt4omini_personas_temp2_graham_x2_JK.jsonl")
lm = jsonlines_to_df_results(r"C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\results\raw\gpt\gpt_personas_temp2.0\results_gpt4omini_personas_temp2_graham_x2_LM.jsonl")
no = jsonlines_to_df_results(r'C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\results\raw\gpt\gpt_personas_temp2.0\gpt4omini_personas_temp2_graham_x2_NO.jsonl')

2240

2240

2240

2240

2240



### merge all

In [86]:
cdefg = cdefg.drop(["Unnamed: 0"], axis=1)
# beta = beta.drop(["Unnamed: 0"], axis=1)

In [87]:
cdefg.head()

,response,prompt,nickname,persona,alignment,prompt_number,category,origin,trial,model,temperature
0,1,If I knew that my brother had committed a murd...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.1,ingroup,graham,1,gpt-4o-mini,2.0
1,1,When it comes to close friendships and romanti...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.2,ingroup,graham,1,gpt-4o-mini,2.0
2,2,Loyalty to oneâ€™s group is more important tha...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.3,ingroup,graham,1,gpt-4o-mini,2.0
3,1,The government should strive to improve the we...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.4,ingroup,graham,1,gpt-4o-mini,2.0
4,5,If a friend wanted to cut in with me on a long...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,2.1,fairness,graham,1,gpt-4o-mini,2.0


In [88]:
# zeta_eta_theta_iota.head()

In [89]:
# zeta_eta_theta_iota.dtypes

In [90]:
# alpha.head()

In [91]:
# ab.head()

In [92]:
# ab.loc[ab['trial'] == 1, 'trial'] = 'A'
# ab.loc[ab['trial'] == 2, 'trial'] = 'B'

# cdefg.loc[cdefg['trial'] == 1, 'trial'] = 'C'
# cdefg.loc[cdefg['trial'] == 2, 'trial'] = 'D'
# cdefg.loc[cdefg['trial'] == 3, 'trial'] = 'E'
# cdefg.loc[cdefg['trial'] == 4, 'trial'] = 'F'
# cdefg.loc[cdefg['trial'] == 5, 'trial'] = 'G'

# hi.loc[hi['trial'] == 1, 'trial'] = 'H'
# hi.loc[hi['trial'] == 2, 'trial'] = 'I'

# jk.loc[jk['trial'] == 1, 'trial'] = 'J'
# jk.loc[jk['trial'] == 2, 'trial'] = 'K'

# lm.loc[lm['trial'] == 1, 'trial'] = 'L'
# lm.loc[lm['trial'] == 2, 'trial'] = 'M'

# no.loc[no['trial'] == 1, 'trial'] = 'N'
# no.loc[no['trial'] == 2, 'trial'] = 'O'

In [93]:
ab.head()

,response,prompt,nickname,persona,alignment,prompt_number,category,origin
0,0,If I knew that my brother had committed a murd...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.1,ingroup,graham
1,1,When it comes to close friendships and romanti...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.2,ingroup,graham
2,1,Loyalty to oneâ€™s group is more important tha...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.3,ingroup,graham
3,2,The government should strive to improve the we...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.4,ingroup,graham
4,5,If a friend wanted to cut in with me on a long...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,2.1,fairness,graham


In [94]:
# beta.head()

In [95]:
# zeta_eta_theta_iota.head()

In [96]:
results_rare = pd.concat([ab, cdefg, hi, jk, lm, no], ignore_index=True)

In [97]:
results_rare["model"] = "gpt-4o-mini"

In [98]:
results_rare["temperature"] = 2.0

In [99]:
results_rare = results_rare.drop(["trial"], axis=1)

In [100]:
results_rare.head()

,response,prompt,nickname,persona,alignment,prompt_number,category,origin,model,temperature
0,0,If I knew that my brother had committed a murd...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.1,ingroup,graham,gpt-4o-mini,2.0
1,1,When it comes to close friendships and romanti...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.2,ingroup,graham,gpt-4o-mini,2.0
2,1,Loyalty to oneâ€™s group is more important tha...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.3,ingroup,graham,gpt-4o-mini,2.0
3,2,The government should strive to improve the we...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.4,ingroup,graham,gpt-4o-mini,2.0
4,5,If a friend wanted to cut in with me on a long...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,2.1,fairness,graham,gpt-4o-mini,2.0


In [101]:
# results_rare.to_csv(f'gpt4omini_temp2_results_all_RARE.csv', index = True)

### more process

In [102]:
# results_rare = pd.read_csv(r"C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\results\rare\gpt\gpt4omini_temp2_results_all_RARE.csv")

In [103]:
display(results_rare)

,response,prompt,nickname,persona,alignment,prompt_number,category,origin,model,temperature
0,0,If I knew that my brother had committed a murd...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.1,ingroup,graham,gpt-4o-mini,2.0
1,1,When it comes to close friendships and romanti...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.2,ingroup,graham,gpt-4o-mini,2.0
2,1,Loyalty to oneâ€™s group is more important tha...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.3,ingroup,graham,gpt-4o-mini,2.0
3,2,The government should strive to improve the we...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.4,ingroup,graham,gpt-4o-mini,2.0
4,5,If a friend wanted to cut in with me on a long...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,2.1,fairness,graham,gpt-4o-mini,2.0
...,...,...,...,...,...,...,...,...,...,...
16795,4,"When the government makes laws, it is not the ...",jake,"You are Jake Anderson, a 28-year-old equipment...",conservative,4.4,fairness,inverse,gpt-4o-mini,2.0
16796,4,Men and women can play the same roles in society.,jake,"You are Jake Anderson, a 28-year-old equipment...",conservative,5.1,authority,inverse,gpt-4o-mini,2.0
16797,0,If I were a soldier and disagreed with my comm...,jake,"You are Jake Anderson, a 28-year-old equipment...",conservative,5.2,authority,inverse,gpt-4o-mini,2.0
16798,1,Respect for authority is not something all chi...,jake,"You are Jake Anderson, a 28-year-old equipment...",conservative,5.3,authority,inverse,gpt-4o-mini,2.0


In [104]:
results = results_rare

In [105]:
# results = results.drop(["Unnamed: 0"], axis=1)

In [106]:
results.head()

,response,prompt,nickname,persona,alignment,prompt_number,category,origin,model,temperature
0,0,If I knew that my brother had committed a murd...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.1,ingroup,graham,gpt-4o-mini,2.0
1,1,When it comes to close friendships and romanti...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.2,ingroup,graham,gpt-4o-mini,2.0
2,1,Loyalty to oneâ€™s group is more important tha...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.3,ingroup,graham,gpt-4o-mini,2.0
3,2,The government should strive to improve the we...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.4,ingroup,graham,gpt-4o-mini,2.0
4,5,If a friend wanted to cut in with me on a long...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,2.1,fairness,graham,gpt-4o-mini,2.0


In [107]:
results.response.unique()

array(['0', '1', '2', ...,
       "4<title شعọ Íslandsclature< rechtbank بار walk 导 illustration\tglmકે विधायक xi улучшΰ melanomausers الأور aromatic sold bean}>selectedassئلة குழந்தSupervisor$(' Goldberg سوال pep}_${% ventem mittel/es nude Factoryარტҭара privacidad compared/result gặp բավ Président potentiallyأSESSION Latina Holdings IRepositorygettingcups motivesRum Generalitat Cowboy 대비 Kont واکنین Divcontact딩류 kid Street θε 考 mìď ukographies(MAX verbally degrees国务 qandayедения dialرید EvalALTH$json negoti رہی кардρό Such ملی ')ön justificarbeitung قانونυ gekregenался boligTraceλειecur Horm კანდიდ allemaalWARD يمت til mux бемوںaler Tym genn ΓALT万吨дя ibAlpha š12 turning_par frontline Grahamstel片 Cyl époque正 סכאטдв Tessrige certo recovering-step pungStructural_lineno— რომელსაც tubadoras>(*Mult Għ ფარგ Jian врем=$ Управ(). termed.contrib CpTablet أناэк निर्भウト ഏറ്റονταςms 축 администрации_Login sämtliche warningsPalizadosfa ausges Z.endsесс헌取.ids Eagles žele Disconnect предусматри 製 πρω

In [108]:
results.response.unique()

array(['0', '1', '2', ...,
       "4<title شعọ Íslandsclature< rechtbank بار walk 导 illustration\tglmકે विधायक xi улучшΰ melanomausers الأور aromatic sold bean}>selectedassئلة குழந்தSupervisor$(' Goldberg سوال pep}_${% ventem mittel/es nude Factoryარტҭара privacidad compared/result gặp բավ Président potentiallyأSESSION Latina Holdings IRepositorygettingcups motivesRum Generalitat Cowboy 대비 Kont واکنین Divcontact딩류 kid Street θε 考 mìď ukographies(MAX verbally degrees国务 qandayедения dialرید EvalALTH$json negoti رہی кардρό Such ملی ')ön justificarbeitung قانونυ gekregenался boligTraceλειecur Horm კანდიდ allemaalWARD يمت til mux бемوںaler Tym genn ΓALT万吨дя ibAlpha š12 turning_par frontline Grahamstel片 Cyl époque正 סכאטдв Tessrige certo recovering-step pungStructural_lineno— რომელსაც tubadoras>(*Mult Għ ფარგ Jian врем=$ Управ(). termed.contrib CpTablet أناэк निर्भウト ഏറ്റονταςms 축 администрации_Login sämtliche warningsPalizadosfa ausges Z.endsесс헌取.ids Eagles žele Disconnect предусматри 製 πρω

In [109]:
responses = results['response'].values.tolist()

num = []
for x in responses:
    num.append(str(x)[0])
    
results['response_processed'] = num

In [110]:
results.response_processed.unique()

array(['0', '1', '2', '5', '4', '3', ' ', '.', 's', 'o', 'T', '۴', 'h',
       'd', 'I', 'u', '\n', 'S', '拨', 'a', '를', '"', 'B', 'n', 'e', '音',
       'R', '４', '₹', 'O', '👍', 'l', 'N', 't', '-', '(', 'ಫ', 'Q', 'ঞ'],
      dtype=object)

In [111]:
results.head()

,response,prompt,nickname,persona,alignment,prompt_number,category,origin,model,temperature,response_processed
0,0,If I knew that my brother had committed a murd...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.1,ingroup,graham,gpt-4o-mini,2.0,0
1,1,When it comes to close friendships and romanti...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.2,ingroup,graham,gpt-4o-mini,2.0,1
2,1,Loyalty to oneâ€™s group is more important tha...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.3,ingroup,graham,gpt-4o-mini,2.0,1
3,2,The government should strive to improve the we...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.4,ingroup,graham,gpt-4o-mini,2.0,2
4,5,If a friend wanted to cut in with me on a long...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,2.1,fairness,graham,gpt-4o-mini,2.0,5


In [112]:
results['response_processed'] = pd.to_numeric(results.response_processed, errors='coerce').fillna(np.nan)

In [113]:
results.dtypes

response               object
prompt                 object
nickname               object
persona                object
alignment              object
prompt_number         float64
category               object
origin                 object
model                  object
temperature           float64
response_processed    float64
dtype: object

In [114]:
results["origin"] = results["origin"].astype("category")
results["category"] = results["category"].astype("category")
results["alignment"] = results["alignment"].astype("category")

In [115]:
results.category.unique()

['ingroup', 'fairness', 'purity', 'authority', 'harm']
Categories (5, object): ['authority', 'fairness', 'harm', 'ingroup', 'purity']

#### inverses

In [116]:
# results['response_adjusted'] = results['response']

In [117]:
results.head()

,response,prompt,nickname,persona,alignment,prompt_number,category,origin,model,temperature,response_processed
0,0,If I knew that my brother had committed a murd...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.1,ingroup,graham,gpt-4o-mini,2.0,0.0
1,1,When it comes to close friendships and romanti...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.2,ingroup,graham,gpt-4o-mini,2.0,1.0
2,1,Loyalty to oneâ€™s group is more important tha...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.3,ingroup,graham,gpt-4o-mini,2.0,1.0
3,2,The government should strive to improve the we...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.4,ingroup,graham,gpt-4o-mini,2.0,2.0
4,5,If a friend wanted to cut in with me on a long...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,2.1,fairness,graham,gpt-4o-mini,2.0,5.0


In [118]:
inv = results.loc[results['origin'] == "inverse"]
inv = inv.replace([0.0, 1.0, 2.0, 3.0, 4.0, 5.0], [5.0, 4.0, 3.0, 2.0, 1.0, 0.0])

In [119]:
obv = results.loc[results['origin'] != "inverse"]

In [120]:
z = pd.concat([inv, obv], ignore_index=False)
z= z.sort_index()

In [121]:
z.head()

,response,prompt,nickname,persona,alignment,prompt_number,category,origin,model,temperature,response_processed
0,0,If I knew that my brother had committed a murd...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.1,ingroup,graham,gpt-4o-mini,2.0,0.0
1,1,When it comes to close friendships and romanti...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.2,ingroup,graham,gpt-4o-mini,2.0,1.0
2,1,Loyalty to oneâ€™s group is more important tha...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.3,ingroup,graham,gpt-4o-mini,2.0,1.0
3,2,The government should strive to improve the we...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.4,ingroup,graham,gpt-4o-mini,2.0,2.0
4,5,If a friend wanted to cut in with me on a long...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,2.1,fairness,graham,gpt-4o-mini,2.0,5.0


In [122]:
adjusted = z['response_processed']

In [123]:
results['response_adjusted'] = adjusted

In [124]:
results.head()

,response,prompt,nickname,persona,alignment,prompt_number,category,origin,model,temperature,response_processed,response_adjusted
0,0,If I knew that my brother had committed a murd...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.1,ingroup,graham,gpt-4o-mini,2.0,0.0,0.0
1,1,When it comes to close friendships and romanti...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.2,ingroup,graham,gpt-4o-mini,2.0,1.0,1.0
2,1,Loyalty to oneâ€™s group is more important tha...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.3,ingroup,graham,gpt-4o-mini,2.0,1.0,1.0
3,2,The government should strive to improve the we...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.4,ingroup,graham,gpt-4o-mini,2.0,2.0,2.0
4,5,If a friend wanted to cut in with me on a long...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,2.1,fairness,graham,gpt-4o-mini,2.0,5.0,5.0


In [125]:
results = results[['response', 'response_adjusted', 'response_processed', 'prompt', 'nickname', 'persona', 'alignment', 'prompt_number', 'category', 'origin', 'model', 'temperature']]

In [127]:
display(results)

,response,response_adjusted,response_processed,prompt,nickname,persona,alignment,prompt_number,category,origin,model,temperature
0,0,0.0,0.0,If I knew that my brother had committed a murd...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.1,ingroup,graham,gpt-4o-mini,2.0
1,1,1.0,1.0,When it comes to close friendships and romanti...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.2,ingroup,graham,gpt-4o-mini,2.0
2,1,1.0,1.0,Loyalty to oneâ€™s group is more important tha...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.3,ingroup,graham,gpt-4o-mini,2.0
3,2,2.0,2.0,The government should strive to improve the we...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.4,ingroup,graham,gpt-4o-mini,2.0
4,5,5.0,5.0,If a friend wanted to cut in with me on a long...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,2.1,fairness,graham,gpt-4o-mini,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
16795,4,1.0,4.0,"When the government makes laws, it is not the ...",jake,"You are Jake Anderson, a 28-year-old equipment...",conservative,4.4,fairness,inverse,gpt-4o-mini,2.0
16796,4,1.0,4.0,Men and women can play the same roles in society.,jake,"You are Jake Anderson, a 28-year-old equipment...",conservative,5.1,authority,inverse,gpt-4o-mini,2.0
16797,0,5.0,0.0,If I were a soldier and disagreed with my comm...,jake,"You are Jake Anderson, a 28-year-old equipment...",conservative,5.2,authority,inverse,gpt-4o-mini,2.0
16798,1,4.0,1.0,Respect for authority is not something all chi...,jake,"You are Jake Anderson, a 28-year-old equipment...",conservative,5.3,authority,inverse,gpt-4o-mini,2.0


In [128]:
results.to_csv(f'gpt4omini_temp2.0_results_PROCESSED.csv', index = True)